<a href="https://colab.research.google.com/github/GB1111/Coursera_Capstone/blob/master/PuneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [33]:
path = 'https://raw.githubusercontent.com/GB1111/Coursera_Capstone/master/pune_data.csv'
pune_data = pd.read_csv(path)
pune_data.head()

,Postal Code,Place Name,County,Community,Latitude,Longitude,Accuracy
0,410301,Khandala,Pune,Maval,18.7589,73.3694,4
1,410302,R P T S Khandala,Pune,Maval,18.7589,73.3694,1
2,410401,Lonavala,Pune,Maval,18.7528,73.4057,4
3,410401,Kusgaon BK,Pune,Maval,18.7528,73.4057,3
4,410401,Lonavala Bazar,Pune,Maval,18.7528,73.4057,3


In [34]:
pune_data = pune_data[['Postal Code', 'Community', 'Place Name', 'Latitude', 'Longitude']]
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [35]:
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [36]:
print('The dataframe has {} communities and {} places.'.format(
        len(pune_data['Community'].unique()),
        pune_data.shape[0]
    )
)

The dataframe has 20 communities and 791 places.


In [37]:
pune_data.groupby('Community').count()

,Postal Code,Place Name,Latitude,Longitude
Community,,,,
Ambegaon,43,43,43,43
Baramati,56,56,56,56
Bhor,69,69,69,69
Daund,52,52,52,52
Haveli,76,76,76,76
Indapur,50,50,50,50
Junnar,68,68,68,68
Khed,68,68,68,68
Maval,38,38,38,38


In [53]:
CLIENT_ID = 'NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO' # your Foursquare ID
CLIENT_SECRET = 'QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO
CLIENT_SECRET:QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS


In [54]:
pune_data.loc[0, 'Place Name']

'Khandala'

In [55]:
place_latitude = pune_data.loc[0, 'Latitude'] # neighborhood latitude value
place_longitude = pune_data.loc[0, 'Longitude'] # neighborhood longitude value

place_name = pune_data.loc[0, 'Place Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(place_name, 
                                                               place_latitude, 
                                                               place_longitude))

Latitude and longitude values of Khandala are 18.7589, 73.3694.


In [56]:
LIMIT = 100
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    place_latitude, 
    place_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NJDMRKKIXGRVKUPRLI4AJDJHDRI5J303R5JNN2FW32HWCSDO&client_secret=QWQYLF3D1BM2T2UOYOVT02KUOS3NANJPU4NIQXU1STJ3ETHS&v=20180605&ll=18.7589,73.3694&radius=500&limit=100'

In [0]:
import requests

In [0]:

results = requests.get(url).json()

In [0]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [62]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: ignored

In [47]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
pune_venues = getNearbyVenues(names=pune_data['Place Name'],
                                   latitudes=pune_data['Latitude'],
                                   longitudes=pune_data['Longitude']
                                 )
print(pune_venues.shape)
pune_venues.head()


Khandala
R P T S Khandala
Lonavala
Kusgaon BK
Lonavala Bazar
Ambavane
Ins Shivaji Lonavale
Kurwande
Kaivalyadham
Karanjgaon
Kamshet
Karunj
Malawali
Karla
Takave KH
Vehergaon
Nane
Pawananagar
Shivali
Bhose
Chakan
Mahalunge Ingale
Vasuli
Waki BK
Ambethan
Vadgaon Ghenanad
Khalumbre
Shive
Rase
Pimpalgaon Tarfe Chakan
Kuruli
Koyali
Nighoje
Kalus
Gadad
Bahul
Rajur
Parunde
Junnar
Tambe
Devale
Inglun
Khanapur
Vadaj
Anjanavale
Kusur
Khangaon
Amboli
Aptale
Yenere
Khamgaon
Khadkumbe(Exptl)
Ballalwadi
Manikdoh
Chavand
Nimgiri
Godre
Nirgude
Utchil
Manchar
Landewadi
Chandoli BK
Vadgaon Kashimbeg
Pimpalgaon Mahalunge
Narodi
Ozar
Khodad
Yedgaon
Pimparkhed
Pargaon Tarfe Ala
Ranjani
Valati
Narayangaon
Nimgaon Sava
Khamundi
Pimpri Pendhar
Jambhut
Hivare BK
Amboli
Vaphgaon
Pimpri BK
Rajgurunagar
Palu
Donda
Nimgaon Khed
Chinchoshi
Pait
Yeniye BK
Kurkundi
Varude
Kude BK
Sandbhorwadi(Exptl)
Kiwale
Kanhersar
Retwadi
Shiroli
Gulani
Ghotawadi
Dawadi
Talegaon Dabhade
Dhamane
Urse
Chandkhed
Adhale BK
Bebedhol
Ord

KeyError: ignored